In [1]:
import tifffile as tiff
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300
import os, shutil
import glob
import numpy as np
from pathlib import Path

Activate cellpose environment

In [2]:
# activate cellpose-env
# to use GPU, first check on the terminal: 
!nvcc --version 
!nvidia-smi
# then turn use GPU to True

from cellpose import core, utils, io, models, metrics, plot
from cellpose.io import imread
from glob import glob

use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Mar_28_02:30:10_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.4, V12.4.131
Build cuda_12.4.r12.4/compiler.34097967_0
Thu May 22 23:53:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 552.22                 Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Quadro RTX 5000              WDDM  |   00000000:D8:00.0 Off |                  Off |
| 33%

c:\Users\fronte\.conda\envs\cellpose\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


>>> GPU activated? YES


Nuclear segmentation

In [2]:
# CHANGE MODEL_PATH, CHANNELS AND DIAMETER AS REQUIRED
# ADD INPUT FOLDER WHERE CROPPED TRACK IMAGES ARE

# Setup logging for Cellpose
io.logger_setup()

### MODEL
# Custom model path
# model_path = "H:/PROJECTS-03/Feyza/segmentation/cellpose_nuclei_membrane/training_data_05z/models/CP_20241007_h2bxncad" #membrane (w nucleus)
model_path = "H:/PROJECTS-03/Agnese/cellpose training/2d/models/nuclei_h2b" #nucleus

model = models.CellposeModel(gpu=True, pretrained_model=model_path)
### PARAMETERS
# Define channels
# channels = [[1, 2]]  # cytoplasm: 1, nucleus: 2
channels = [[0,0]] # only nucleus: if multi-channel image 3=nucleus is the third channel in the image. If there is only the nucleus (gray scale) is [0,0]

# Segmentation parameters
# diameter = 25  # in pixels for membrane
diameter = 17 # in pixels for nucleus
cellprob_threshold = -1

# Input folder containing images
input_folder = Path("H:/PROJECTS-03/Agnese/20240913_modERK_h2bcherry_SU5402-trial/20240913_modERK_mcherry_SU5402/mask")
output_folder = input_folder / "denoised_segmentation_masks_nuclei"
output_folder.mkdir(exist_ok=True)  # Create folder if it doesn't exist

# Process all files in the folder
for img_file in input_folder.glob("*.tif"):
    print(f"Processing file: {img_file.name}")

    # Read image
    imgs = imread(img_file)
    print(f'Loaded image {img_file.name} with shape: {imgs.shape} and data type {imgs.dtype}')

    # Generate output file name
    output_file_base = output_folder / img_file.stem  # Use the base name without extension

    # USE cellposeDenoiseModel FOR DENOISING THE IMAGE
    from cellpose import denoise
    dn = denoise.DenoiseModel (model_type='denoise_cyto3', gpu=True, chan2=True)
    imgs = dn.eval(imgs, channels=[0,0], diameter=17.)

    # Perform segmentation
    masks, flows, styles = model.eval(
        imgs,
        diameter=diameter,
        channels=channels,
        cellprob_threshold=cellprob_threshold,
        do_3D=False,
        anisotropy=1.5,
        min_size=-1,
    )

    # Save output to *_seg.npy
    seg_file = str(output_file_base) + "_seg"
    io.masks_flows_to_seg(imgs, masks, flows, seg_file, channels=channels)

    # Save output masks to tiffs/pngs or txt files for ImageJ
    io.save_masks(
        imgs,
        masks,
        flows,
        str(output_file_base),
        channels=channels,
        png=False,  # Save masks as PNGs
        tif=True,  # Save masks as TIFFs
        save_txt=True,  # Save txt outlines for ImageJ
        save_flows=False,  # Save flows as TIFFs
        save_outlines=False,  # Save outlines as TIFFs
        save_mpl=False,  # Make matplotlib fig to view (WARNING: SLOW W/ LARGE IMAGES)
    )

    print(f"Saved results for {img_file.name} to {output_folder}")

print("All files processed.")

NameError: name 'io' is not defined

Isolate center nuclei

In [8]:
# function using fill holes and remove small masks util from cellpose, then picking the segment in the center of the volume and saving binarized images

# could add remove edge masks to this in the future, after making sure volumes are big enough to keep center mask away from edges

import glob

def isolate_center_mask(image, min_size=100):
    """
    Process a 2D TIFF image of masks:
    - Fill holes and remove small masks.
    - Identify the mask at the center of the center slice.
    - Keep only that mask.
    - Binarize the resulting image.
    - Save the processed image.

    Parameters:
        min_size (int): Minimum size of masks to retain during processing.
    """
    
    # Apply pre-processing: fill holes and remove small masks
    image = utils.fill_holes_and_remove_small_masks(image, min_size=min_size)
    
    # Extract the center label
    center_y, center_x = image.shape[0] // 2, image.shape[1] // 2
    central_mask_label = image[center_y, center_x]
    
    # Check if the center pixel is part of a mask
    if central_mask_label == 0:
        raise ValueError("The center of the center slice does not contain a mask.")
    
    # Create a binary mask for the entire volume
    binary_image = (image == central_mask_label).astype(np.uint8)

    return binary_image


def isolate_center_mask_in_folder(input_path, min_size=100):
    """
    Process all 3D TIFF mask files in a folder, isolating the central mask for each,
    and save the processed binary images to a new output folder.

    Parameters:
        input_path (str): Path to the folder containing mask files ending with "_masks".
        min_size (int): Minimum size of masks to retain during processing.
    """
    
    # Find all files ending with "_masks"
    mask_files = glob.glob(os.path.join(input_path, "*_masks.tif"))
    if not mask_files:
        raise ValueError("No mask files ending with '_masks.tif' were found in the input folder.")
    
    # Create an output folder in the parent directory
    parent_dir = os.path.dirname(input_path)
    output_folder = os.path.join(parent_dir, "center_masks")
    os.makedirs(output_folder, exist_ok=True)
    
    # Process each mask file
    for mask_file in mask_files:
        print(f"Processing {mask_file}...")
        # Load the mask volume
        image = tiff.imread(mask_file)
        
        # Process the volume
        try:
            binary_image = isolate_center_mask(image, min_size=min_size)
        except ValueError as e:
            print(f"Skipping {mask_file} due to error: {e}")
            continue
        
        # Save the processed binary volume
        base_name = os.path.basename(mask_file)
        output_path = os.path.join(output_folder, base_name.replace("_masks", "_center"))
        tiff.imwrite(output_path, binary_image, dtype=np.uint8)
        print(f"Saved processed file to {output_path}")
    
    print(f"Processing complete. Files saved to {output_folder}")

In [13]:
#nuclei
input_path = 'H:/PROJECTS-03/Agnese/20240808_modERK_h2bcherry-bleached/check_injection/neigh/neigh1/nuclei/denoised_segmentation_masks_nuclei/'

isolate_center_mask_in_folder(input_path, min_size=1)

Processing H:/PROJECTS-03/Agnese/20240808_modERK_h2bcherry-bleached/check_injection/neigh/neigh1/nuclei/denoised_segmentation_masks_nuclei\Cell_7_T_0000_cp_masks.tif...
Saved processed file to H:/PROJECTS-03/Agnese/20240808_modERK_h2bcherry-bleached/check_injection/neigh/neigh1/nuclei/denoised_segmentation_masks_nuclei\center_masks\Cell_7_T_0000_cp_center.tif
Processing H:/PROJECTS-03/Agnese/20240808_modERK_h2bcherry-bleached/check_injection/neigh/neigh1/nuclei/denoised_segmentation_masks_nuclei\Cell_7_T_0001_cp_masks.tif...
Saved processed file to H:/PROJECTS-03/Agnese/20240808_modERK_h2bcherry-bleached/check_injection/neigh/neigh1/nuclei/denoised_segmentation_masks_nuclei\center_masks\Cell_7_T_0001_cp_center.tif
Processing H:/PROJECTS-03/Agnese/20240808_modERK_h2bcherry-bleached/check_injection/neigh/neigh1/nuclei/denoised_segmentation_masks_nuclei\Cell_7_T_0002_cp_masks.tif...
Saved processed file to H:/PROJECTS-03/Agnese/20240808_modERK_h2bcherry-bleached/check_injection/neigh/neig

Plotting

In [2]:
#fgf and modERK tracking plot with treshold (Otsu) and excluding nucleus pixels for the fgf calculation

import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk
from skimage.filters import threshold_otsu  # Import Otsu's method

def process_and_plot_2D_fgf(input_nuclear_mask_folder, input_fluorescence_folder_1, input_fluorescence_folder_2, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=3):
    os.makedirs(output_plot_folder, exist_ok=True)

    nuclear_mask_files = sorted([f for f in os.listdir(input_nuclear_mask_folder) if f.endswith(".tif")])
    fluorescence_files_1 = sorted([f for f in os.listdir(input_fluorescence_folder_1) if f.endswith(".tif")])
    fluorescence_files_2 = sorted([f for f in os.listdir(input_fluorescence_folder_2) if f.endswith(".tif")])
    
    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")
    
    grouped_files = {}

    for nuc_mask_file in nuclear_mask_files:
        match = nuclear_pattern.match(nuc_mask_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["nuclear_mask"] = nuc_mask_file

    for fluo_file in fluorescence_files_1:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_1"] = fluo_file
    
    for fluo_file in fluorescence_files_2:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_2"] = fluo_file

    for cell_id, timepoint_dict in grouped_files.items():
        valid_timepoints = sorted([tp for tp in timepoint_dict if all(key in timepoint_dict[tp] for key in ["nuclear_mask", "fluorescence_1", "fluorescence_2"])])
        
        timepoints, log2_ratios, fgf_mean_intensities = [], [], []
        
        for timepoint in valid_timepoints:
            files = timepoint_dict[timepoint]
            nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
            fluorescence_1 = imread(os.path.join(input_fluorescence_folder_1, files["fluorescence_1"]))
            fluorescence_2 = imread(os.path.join(input_fluorescence_folder_2, files["fluorescence_2"]))
            
            selem = disk(dilation_erosion_pixels)
            eroded_nuclear_mask = binary_erosion(nuclear_mask, selem)
            dilated_nuclear_mask = binary_dilation(nuclear_mask, selem)
            cytoplasm_mask = dilated_nuclear_mask & ~nuclear_mask
            
            nucleus_intensity = fluorescence_1[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            cytoplasm_intensity = fluorescence_1[cytoplasm_mask].mean() if np.any(cytoplasm_mask) else 0
            
            # Compute Otsu's threshold on the entire fgf image
            otsu_threshold = threshold_otsu(fluorescence_2)

            # Select pixels outside the nucleus and above the Otsu threshold
            outside_nucleus_mask = ~nuclear_mask  # Everything not in the nucleus
            high_intensity_mask = fluorescence_2 > otsu_threshold  # Pixels above threshold
            valid_outside_nucleus = outside_nucleus_mask & high_intensity_mask  # Combined mask

            # Calculate mean intensity for fgf outside the nucleus
            mean_intensity_outside_nucleus = fluorescence_2[valid_outside_nucleus].mean() if np.any(valid_outside_nucleus) else 0
            
            if nucleus_intensity > 0 and cytoplasm_intensity > 0:
                log2_ratio = np.log2(cytoplasm_intensity / nucleus_intensity)
                if log2_ratio != 0:
                    timepoints.append(timepoint * 2)
                    log2_ratios.append(log2_ratio)
                    fgf_mean_intensities.append(mean_intensity_outside_nucleus)  # Store new intensity metric
        
        if log2_ratios:
            binned_timepoints = np.array(timepoints)[::bin_size]
            binned_log2_ratios = np.array(log2_ratios)[::bin_size]
            binned_fgf_intensities = np.array(fgf_mean_intensities)[::bin_size]
            
            smoothed_log2_ratios = gaussian_filter1d(binned_log2_ratios, sigma=smoothing_sigma, mode='nearest')
            smoothed_fgf_intensities = gaussian_filter1d(binned_fgf_intensities, sigma=smoothing_sigma, mode='nearest')

            plt.figure(figsize=(8, 6))
            fig, ax1 = plt.subplots()

            ax1.set_xlabel("Time (minutes)")
            ax1.set_ylabel("Log2(C/N) dpERK", color='m')
            ax1.plot(binned_timepoints, smoothed_log2_ratios, marker='.', linestyle='-', color='m')
            ax1.tick_params(axis='y', labelcolor='m')
            
            ax2 = ax1.twinx()
            ax2.set_ylabel("fgf", color='darkblue')
            ax2.plot(binned_timepoints, smoothed_fgf_intensities, marker='.', linestyle='-', color='darkblue')
            ax2.tick_params(axis='y', labelcolor='darkblue')

            fig.tight_layout()
            plt.grid(True)
            plt.savefig(os.path.join(output_plot_folder, f"Cell_{cell_id}_log2_and_fgf.svg"))
            plt.close()
    
    print("Processing and plotting complete!")

input_nuclear_mask_folder = "E:/Agnese/20250207_fgf8a_moderk_h2a_late/20250207_155200_Experiment/mastodon_tracking/all/nuclei/"
input_erk_folder = "E:/Agnese/20250207_fgf8a_moderk_h2a_late/20250207_155200_Experiment/mastodon_tracking/all/erk/"
input_fgf_folder = "E:/Agnese/20250207_fgf8a_moderk_h2a_late/20250207_155200_Experiment/mastodon_tracking/all/fgf/"
output_plot_folder = "E:/Agnese/20250207_fgf8a_moderk_h2a_late/20250207_155200_Experiment/mastodon_tracking/all/"

process_and_plot_2D_fgf(input_nuclear_mask_folder, input_erk_folder, input_fgf_folder, output_plot_folder, dilation_erosion_pixels=2, smoothing_sigma=3, bin_size=3)

Processing and plotting complete!


<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

...

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

<Figure size 2400x1800 with 0 Axes>

In [14]:
#plotting just erk in all the cells

import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.optimize import curve_fit
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk
from skimage.filters import threshold_otsu

def process_and_plot_selected_cells(input_nuclear_mask_folder, input_fluorescence_folder_1, output_plot_folder, selected_cell_ids, cell_groups, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=3):
    os.makedirs(output_plot_folder, exist_ok=True)
    
    nuclear_mask_files = sorted([f for f in os.listdir(input_nuclear_mask_folder) if f.endswith(".tif")])
    fluorescence_files_1 = sorted([f for f in os.listdir(input_fluorescence_folder_1) if f.endswith(".tif")])
    
    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")
    
    grouped_files = {}
    
    for nuc_mask_file in nuclear_mask_files:
        match = nuclear_pattern.match(nuc_mask_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            if cell_id in selected_cell_ids:
                grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["nuclear_mask"] = nuc_mask_file

    for fluo_file in fluorescence_files_1:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            if cell_id in selected_cell_ids:
                grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_1"] = fluo_file
    
    
    max_timepoint = max(tp for cell in grouped_files.values() for tp in cell)
    group_colors = {"anterior": "pink", "mid": "magenta", "posterior": "purple"}
    group_labels = set()

    plt.figure(figsize=(8, 6))
    
    # Find the maximum timepoint across all cells
    for cell_id, timepoint_dict in grouped_files.items():
        valid_timepoints = sorted([tp for tp in timepoint_dict if all(key in timepoint_dict[tp] for key in ["nuclear_mask", "fluorescence_1"])])
        #valid_timepoints = [tp for tp in valid_timepoints if tp*2 < 80]
            
        timepoints, log2_ratios = [], []
        
        for timepoint in valid_timepoints:
            files = timepoint_dict[timepoint]
            nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
            fluorescence_1 = imread(os.path.join(input_fluorescence_folder_1, files["fluorescence_1"]))
            
            selem = disk(dilation_erosion_pixels)
            eroded_nuclear_mask = binary_erosion(nuclear_mask, selem)
            dilated_nuclear_mask = binary_dilation(nuclear_mask, selem)
            cytoplasm_mask = dilated_nuclear_mask & ~nuclear_mask
            
            nucleus_intensity = fluorescence_1[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            cytoplasm_intensity = fluorescence_1[cytoplasm_mask].mean() if np.any(cytoplasm_mask) else 0
            
            if nucleus_intensity > 0 and cytoplasm_intensity > 0:
                log2_ratio = np.log2(cytoplasm_intensity / nucleus_intensity)
                if log2_ratio != 0:
                    timepoints.append(timepoint * 2)
                    log2_ratios.append(log2_ratio)
        
        if log2_ratios:
            binned_timepoints = np.array(timepoints)[::bin_size]
            binned_log2_ratios = np.array(log2_ratios)[::bin_size]
            smoothed_log2_ratios = gaussian_filter1d(binned_log2_ratios, sigma=smoothing_sigma, mode='nearest')
            
            # Shift the timepoints so that the last timepoint is the same across all cells
            shift = max_timepoint - binned_timepoints[-1]
            aligned_timepoints = binned_timepoints + shift
            
            cell_group = cell_groups.get(cell_id, "mid")  # Default to "mid" if not specified
            color = group_colors.get(cell_group, "black")
            
            # Plot using the group name, avoiding the cell ID in the label
            if cell_group not in group_labels:  # Ensure each group is only labeled once
                plt.plot(aligned_timepoints, smoothed_log2_ratios, marker='.', linestyle='-', color=color, label=cell_group)
                group_labels.add(cell_group)
            else:
                plt.plot(aligned_timepoints, smoothed_log2_ratios, marker='.', linestyle='-', color=color)
    
    plt.xlabel("Time (minutes)")
    plt.ylabel("Log2(C/N) dpERK")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_plot_folder, "just_first.svg"))
    plt.close()
    
    print("Processing and plotting complete!")


# Example function call:
input_nuclear_mask_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/nuclei/"
input_erk_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/erk/"
input_fgf_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/fgf/"
output_plot_folder = "E:/Agnese/20250129_h2bxlooping_moderk_memcer_late/tracking_2d_LATE/all/"
selected_cell_ids = [4]
cell_groups = {
    **{cell: "anterior" for cell in [4]},
    **{cell: "mid" for cell in []},
    **{cell: "posterior" for cell in []}
}


process_and_plot_selected_cells(input_nuclear_mask_folder, input_erk_folder, output_plot_folder, selected_cell_ids, cell_groups, dilation_erosion_pixels=2, smoothing_sigma=2, bin_size=1)

Processing and plotting complete!


In [18]:
#plotting just erk NORMALIZED IN Y in all the cells

import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.optimize import curve_fit
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk
from skimage.filters import threshold_otsu

def process_and_plot_selected_cells(input_nuclear_mask_folder, input_fluorescence_folder_1, output_plot_folder, selected_cell_ids, cell_groups, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=3):
    os.makedirs(output_plot_folder, exist_ok=True)
    
    nuclear_mask_files = sorted([f for f in os.listdir(input_nuclear_mask_folder) if f.endswith(".tif")])
    fluorescence_files_1 = sorted([f for f in os.listdir(input_fluorescence_folder_1) if f.endswith(".tif")])
    
    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")
    
    grouped_files = {}
    
    for nuc_mask_file in nuclear_mask_files:
        match = nuclear_pattern.match(nuc_mask_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            if cell_id in selected_cell_ids:
                grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["nuclear_mask"] = nuc_mask_file

    for fluo_file in fluorescence_files_1:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            if cell_id in selected_cell_ids:
                grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_1"] = fluo_file
    
    
    max_timepoint = max(tp for cell in grouped_files.values() for tp in cell)
    group_colors = {"anterior": "plum", "mid": "magenta", "posterior": "teal"}
    group_labels = set()

    plt.figure(figsize=(8, 6))
    
    # Find the maximum timepoint across all cells
    for cell_id, timepoint_dict in grouped_files.items():
        valid_timepoints = sorted([tp for tp in timepoint_dict if all(key in timepoint_dict[tp] for key in ["nuclear_mask", "fluorescence_1"])])
            
        timepoints, log2_ratios = [], []
        
        for timepoint in valid_timepoints:
            files = timepoint_dict[timepoint]
            nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
            fluorescence_1 = imread(os.path.join(input_fluorescence_folder_1, files["fluorescence_1"]))
            
            selem = disk(dilation_erosion_pixels)
            eroded_nuclear_mask = binary_erosion(nuclear_mask, selem)
            dilated_nuclear_mask = binary_dilation(nuclear_mask, selem)
            cytoplasm_mask = dilated_nuclear_mask & ~nuclear_mask
            
            nucleus_intensity = fluorescence_1[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            cytoplasm_intensity = fluorescence_1[cytoplasm_mask].mean() if np.any(cytoplasm_mask) else 0
            
            if nucleus_intensity > 0 and cytoplasm_intensity > 0:
                log2_ratio = np.log2(cytoplasm_intensity / nucleus_intensity)
                if log2_ratio != 0:
                    timepoints.append(timepoint * 2)
                    log2_ratios.append(log2_ratio)
        
        if log2_ratios:
            binned_timepoints = np.array(timepoints)[::bin_size]
            binned_log2_ratios = np.array(log2_ratios)[::bin_size]
            smoothed_log2_ratios = gaussian_filter1d(binned_log2_ratios, sigma=smoothing_sigma, mode='nearest')

            min_val, max_val = smoothed_log2_ratios.min(), smoothed_log2_ratios.max()
            if max_val > min_val:
                normalized_log2_ratios = (smoothed_log2_ratios - min_val) / (max_val - min_val)
            else:
                normalized_log2_ratios = smoothed_log2_ratios
            
            # Shift the timepoints so that the last timepoint is the same across all cells
            shift = max_timepoint - binned_timepoints[-1]
            aligned_timepoints = binned_timepoints + shift
            
            cell_group = cell_groups.get(cell_id, "mid")  # Default to "mid" if not specified
            color = group_colors.get(cell_group, "black")
            
            # Plot using the group name, avoiding the cell ID in the label
            if cell_group not in group_labels:  # Ensure each group is only labeled once
                plt.plot(aligned_timepoints, normalized_log2_ratios, marker='.', linestyle='-', color=color, label=cell_group)
                group_labels.add(cell_group)
            else:
                plt.plot(aligned_timepoints, normalized_log2_ratios, marker='.', linestyle='-', color=color)
    
    plt.xlabel("Time (minutes)")
    plt.ylabel("Log2(C/N) dpERK")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_plot_folder, "S1.svg"))
    plt.close()
    
    print("Processing and plotting complete!")


input_nuclear_mask_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/nuclei/"
input_erk_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/erk/"
input_fgf_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/fgf/"
output_plot_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/"
selected_cell_ids = [10,11,12,13,1,2,3,4]
cell_groups = {
    **{cell: "anterior" for cell in [10,11,12,13]},
    **{cell: "mid" for cell in []},
    **{cell: "posterior" for cell in [1,2,3,4]}
}

process_and_plot_selected_cells(input_nuclear_mask_folder, input_erk_folder, output_plot_folder, selected_cell_ids, cell_groups, dilation_erosion_pixels=2, smoothing_sigma=2, bin_size=2)

Processing and plotting complete!


In [16]:
# average only (no std) just for erk with normalization after averaging
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk

def process_and_plot_selected_cells(input_nuclear_mask_folder, input_fluorescence_folder_1, output_plot_folder, selected_cell_ids, cell_groups, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=3):
    os.makedirs(output_plot_folder, exist_ok=True)
    
    nuclear_mask_files = sorted([f for f in os.listdir(input_nuclear_mask_folder) if f.endswith(".tif")])
    fluorescence_files_1 = sorted([f for f in os.listdir(input_fluorescence_folder_1) if f.endswith(".tif")])
    
    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")
    
    grouped_files = {}
    
    for nuc_mask_file in nuclear_mask_files:
        match = nuclear_pattern.match(nuc_mask_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            if cell_id in selected_cell_ids:
                grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["nuclear_mask"] = nuc_mask_file

    for fluo_file in fluorescence_files_1:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            if cell_id in selected_cell_ids:
                grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_1"] = fluo_file
    
    max_timepoint = max(tp for cell in grouped_files.values() for tp in cell)
    group_colors = {"anterior": "plum", "mid": "magenta", "posterior": "teal"}

    plt.figure(figsize=(8, 6))
    
    # Store log2 ratios for each group
    group_log2_ratios = {"anterior": [], "mid": [], "posterior": []}
    
    for cell_id, timepoint_dict in grouped_files.items():
        valid_timepoints = sorted([tp for tp in timepoint_dict if all(key in timepoint_dict[tp] for key in ["nuclear_mask", "fluorescence_1"])])
            
        timepoints, log2_ratios = [], []
        
        for timepoint in valid_timepoints:
            files = timepoint_dict[timepoint]
            nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
            fluorescence_1 = imread(os.path.join(input_fluorescence_folder_1, files["fluorescence_1"]))
            
            selem = disk(dilation_erosion_pixels)
            eroded_nuclear_mask = binary_erosion(nuclear_mask, selem)
            dilated_nuclear_mask = binary_dilation(nuclear_mask, selem)
            cytoplasm_mask = dilated_nuclear_mask & ~nuclear_mask
            
            nucleus_intensity = fluorescence_1[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            cytoplasm_intensity = fluorescence_1[cytoplasm_mask].mean() if np.any(cytoplasm_mask) else 0
            
            if nucleus_intensity > 0 and cytoplasm_intensity > 0:
                log2_ratio = np.log2(cytoplasm_intensity / nucleus_intensity)
                if log2_ratio != 0:
                    timepoints.append(timepoint * 2)
                    log2_ratios.append(log2_ratio)
        
        if log2_ratios:
            binned_timepoints = np.array(timepoints)[::bin_size]
            binned_log2_ratios = np.array(log2_ratios)[::bin_size]
            smoothed_log2_ratios = gaussian_filter1d(binned_log2_ratios, sigma=smoothing_sigma, mode='nearest')

            shift = max_timepoint - binned_timepoints[-1]
            aligned_timepoints = binned_timepoints + shift

            cell_group = cell_groups.get(cell_id, "mid")
            group_log2_ratios[cell_group].append(smoothed_log2_ratios)
    
    for group, log2_ratios_list in group_log2_ratios.items():
        if log2_ratios_list:
            max_length = max(len(ratios) for ratios in log2_ratios_list)
            padded_log2_ratios = [
                np.pad(ratios, (max_length - len(ratios), 0), mode='constant', constant_values=np.nan)
                if len(ratios) < max_length else ratios[-max_length:]
                for ratios in log2_ratios_list
            ]
            
            stacked_log2_ratios = np.vstack(padded_log2_ratios)
            avg_log2_ratios = np.nanmean(stacked_log2_ratios, axis=0)

            min_val, max_val = avg_log2_ratios.min(), avg_log2_ratios.max()
            normalized_avg_log2_ratios = (avg_log2_ratios - min_val) / (max_val - min_val) if max_val > min_val else avg_log2_ratios

            aligned_timepoints_resampled = np.linspace(aligned_timepoints[0], aligned_timepoints[-1], len(normalized_avg_log2_ratios))

            plt.plot(aligned_timepoints_resampled, normalized_avg_log2_ratios, label=f"{group} (avg)", color=group_colors.get(group, 'gray'))
    
    plt.xlabel("Time (minutes)")
    plt.ylabel("Log2(C/N) dpERK (Normalized)")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_plot_folder, "all_ant_mean.svg"))
    plt.close()
    
    print("Processing and plotting complete!")

input_nuclear_mask_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/nuclei/"
input_erk_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/erk/"
input_fgf_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/fgf/"
output_plot_folder = "E:/Agnese/20250219_loopingxh2b_moderk_memcer/20250219_144705_Experiment/tracking/all/"
selected_cell_ids = [10,11,12,13,7,8,9,29,30,31,32]
cell_groups = {
    **{cell: "anterior" for cell in [10,11,12,13]},
    **{cell: "mid" for cell in [29,30,31,32,7,8,9]},
    **{cell: "posterior" for cell in []}
}

process_and_plot_selected_cells(input_nuclear_mask_folder, input_erk_folder, output_plot_folder, selected_cell_ids, cell_groups, dilation_erosion_pixels=2, smoothing_sigma=2, bin_size=3)

Processing and plotting complete!


In [ ]:
#average + std just for erk with normalization after averaging
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.optimize import curve_fit
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk
from skimage.filters import threshold_otsu

def process_and_plot_selected_cells(input_nuclear_mask_folder, input_fluorescence_folder_1, output_plot_folder, selected_cell_ids, cell_groups, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=3):
    os.makedirs(output_plot_folder, exist_ok=True)
    
    nuclear_mask_files = sorted([f for f in os.listdir(input_nuclear_mask_folder) if f.endswith(".tif")])
    fluorescence_files_1 = sorted([f for f in os.listdir(input_fluorescence_folder_1) if f.endswith(".tif")])
    
    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")
    
    grouped_files = {}
    
    for nuc_mask_file in nuclear_mask_files:
        match = nuclear_pattern.match(nuc_mask_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            if cell_id in selected_cell_ids:
                grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["nuclear_mask"] = nuc_mask_file

    for fluo_file in fluorescence_files_1:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            if cell_id in selected_cell_ids:
                grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_1"] = fluo_file
    
    max_timepoint = max(tp for cell in grouped_files.values() for tp in cell)
    group_colors = {"anterior": "pink", "mid": "magenta", "posterior": "purple"}
    group_labels = set()

    plt.figure(figsize=(8, 6))
    
    # Store log2 ratios for each group
    group_log2_ratios = {"anterior": [], "mid": [], "posterior": []}
    
    # Find the maximum timepoint across all cells
    for cell_id, timepoint_dict in grouped_files.items():
        valid_timepoints = sorted([tp for tp in timepoint_dict if all(key in timepoint_dict[tp] for key in ["nuclear_mask", "fluorescence_1"])])
            
        timepoints, log2_ratios = [], []
        
        for timepoint in valid_timepoints:
            files = timepoint_dict[timepoint]
            nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
            fluorescence_1 = imread(os.path.join(input_fluorescence_folder_1, files["fluorescence_1"]))
            
            selem = disk(dilation_erosion_pixels)
            eroded_nuclear_mask = binary_erosion(nuclear_mask, selem)
            dilated_nuclear_mask = binary_dilation(nuclear_mask, selem)
            cytoplasm_mask = dilated_nuclear_mask & ~nuclear_mask
            
            nucleus_intensity = fluorescence_1[eroded_nuclear_mask].mean() if np.any(eroded_nuclear_mask) else 0
            cytoplasm_intensity = fluorescence_1[cytoplasm_mask].mean() if np.any(cytoplasm_mask) else 0
            
            if nucleus_intensity > 0 and cytoplasm_intensity > 0:
                log2_ratio = np.log2(cytoplasm_intensity / nucleus_intensity)
                if log2_ratio != 0:
                    timepoints.append(timepoint * 2)
                    log2_ratios.append(log2_ratio)
        
        if log2_ratios:
            binned_timepoints = np.array(timepoints)[::bin_size]
            binned_log2_ratios = np.array(log2_ratios)[::bin_size]
            smoothed_log2_ratios = gaussian_filter1d(binned_log2_ratios, sigma=smoothing_sigma, mode='nearest')

            # Shift the timepoints so that the last timepoint is the same across all cells
            shift = max_timepoint - binned_timepoints[-1]
            aligned_timepoints = binned_timepoints + shift

            cell_group = cell_groups.get(cell_id, "mid")  # Default to "mid" if not specified
            group_log2_ratios[cell_group].append(smoothed_log2_ratios)
    
    # Plotting the average and std for each group
    for group, log2_ratios_list in group_log2_ratios.items():
        if log2_ratios_list:
            # Find the maximum length across all log2 ratios in the group
            max_length = max(len(ratios) for ratios in log2_ratios_list)
            
            # Pad or truncate each array to match the max length
            padded_log2_ratios = [
                np.pad(ratios, (max_length - len(ratios), 0), mode='constant', constant_values=np.nan)
                if len(ratios) < max_length else ratios[-max_length:]
                for ratios in log2_ratios_list
                ]
            
            # Stack the log2 ratios of all cells in the group
            stacked_log2_ratios = np.vstack(padded_log2_ratios)

            # Average and std computation across cells (before normalization)
            avg_log2_ratios = np.nanmean(stacked_log2_ratios, axis=0)
            std_log2_ratios = np.nanstd(stacked_log2_ratios, axis=0)

            # Normalize in Y direction (after averaging and std)
            min_val, max_val = avg_log2_ratios.min(), avg_log2_ratios.max()
            if max_val > min_val:
                normalized_avg_log2_ratios = (avg_log2_ratios - min_val) / (max_val - min_val)
                normalized_std_log2_ratios = (std_log2_ratios) / (max_val - min_val)
            else:
                normalized_avg_log2_ratios = avg_log2_ratios
                normalized_std_log2_ratios = std_log2_ratios

            # Resample aligned_timepoints to match the length of normalized_avg_log2_ratios
            aligned_timepoints_resampled = np.linspace(aligned_timepoints[0], aligned_timepoints[-1], len(normalized_avg_log2_ratios))

            # Plot the group
            plt.plot(aligned_timepoints_resampled, normalized_avg_log2_ratios, label=f"{group} (avg)")
            plt.fill_between(aligned_timepoints_resampled, normalized_avg_log2_ratios - normalized_std_log2_ratios, normalized_avg_log2_ratios + normalized_std_log2_ratios, alpha=0.2)
    
    plt.xlabel("Time (minutes)")
    plt.ylabel("Log2(C/N) dpERK (Normalized)")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_plot_folder, "cells.svg"))
    plt.close()
    
    print("Processing and plotting complete!")

# Example function call:
input_nuclear_mask_folder = "E:/Agnese/20250207_fgf8a_moderk_h2a_late/20250207_155200_Experiment/mastodon_tracking/all/nuclei/"
input_erk_folder = "E:/Agnese/20250207_fgf8a_moderk_h2a_late/20250207_155200_Experiment/mastodon_tracking/all/erk/"
input_fgf_folder = "E:/Agnese/20250207_fgf8a_moderk_h2a_late/20250207_155200_Experiment/mastodon_tracking/all/fgf/"
output_plot_folder = "E:/Agnese/20250207_fgf8a_moderk_h2a_late/20250207_155200_Experiment/mastodon_tracking/all/"
#selected_cell_ids = [8,9,13,20,21,22]
selected_cell_ids = [1,2,3,4,6,7,13,14,11,12]
cell_groups = {
    **{cell: "anterior" for cell in [11,12,13,14]},
    **{cell: "mid" for cell in []},
    **{cell: "posterior" for cell in [1,2,3,4,6,7]}
}

process_and_plot_selected_cells(input_nuclear_mask_folder, input_erk_folder, output_plot_folder, selected_cell_ids, cell_groups, dilation_erosion_pixels=2, smoothing_sigma=3, bin_size=3)

Processing and plotting complete!


In [14]:
#fgf for all the cells
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from skimage.io import imread
from skimage.morphology import binary_dilation, binary_erosion, disk
from skimage.filters import threshold_otsu

def process_and_plot_fgf(input_nuclear_mask_folder, input_fluorescence_folder_2, output_plot_folder, selected_cell_ids, cell_groups, dilation_erosion_pixels=2, smoothing_sigma=5, bin_size=3):
    os.makedirs(output_plot_folder, exist_ok=True)

    nuclear_mask_files = sorted([f for f in os.listdir(input_nuclear_mask_folder) if f.endswith(".tif")])
    fluorescence_files_2 = sorted([f for f in os.listdir(input_fluorescence_folder_2) if f.endswith(".tif")])
    
    nuclear_pattern = re.compile(r"Cell_(\d+)_T_(\d+)_cp_center\.tif")
    fluorescence_pattern = re.compile(r"Cell_(\d+)_T_(\d+)\.tif")
    
    grouped_files = {}

    # Group files by cell_id and timepoint
    for nuc_mask_file in nuclear_mask_files:
        match = nuclear_pattern.match(nuc_mask_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            if cell_id in selected_cell_ids:
                grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["nuclear_mask"] = nuc_mask_file

    for fluo_file in fluorescence_files_2:
        match = fluorescence_pattern.match(fluo_file)
        if match:
            cell_id, timepoint = int(match.group(1)), int(match.group(2))
            if cell_id in selected_cell_ids:
                grouped_files.setdefault(cell_id, {}).setdefault(timepoint, {})["fluorescence_2"] = fluo_file

    # Find the maximum timepoint across all cells
    max_timepoint = max(tp for cell in grouped_files.values() for tp in cell)

    group_colors = {"anterior": "pink", "mid": "magenta", "posterior": "purple"}
    group_labels = set()

    plt.figure(figsize=(8, 6))

    # Process each selected cell
    for cell_id, timepoint_dict in grouped_files.items():
        valid_timepoints = sorted([tp for tp in timepoint_dict if all(key in timepoint_dict[tp] for key in ["nuclear_mask", "fluorescence_2"])])
        
        timepoints, fgf_mean_intensities = [], []
        
        for timepoint in valid_timepoints:
            files = timepoint_dict[timepoint]
            nuclear_mask = imread(os.path.join(input_nuclear_mask_folder, files["nuclear_mask"])).astype(bool)
            fluorescence_2 = imread(os.path.join(input_fluorescence_folder_2, files["fluorescence_2"]))

            # Otsu's threshold to select pixels outside the nucleus for fgf calculation
            otsu_threshold = threshold_otsu(fluorescence_2)
            outside_nucleus_mask = ~nuclear_mask
            high_intensity_mask = fluorescence_2 > otsu_threshold
            valid_outside_nucleus = outside_nucleus_mask & high_intensity_mask

            mean_intensity_outside_nucleus = fluorescence_2[valid_outside_nucleus].mean() if np.any(valid_outside_nucleus) else 0

            timepoints.append(timepoint * 2)  # Convert time to minutes
            fgf_mean_intensities.append(mean_intensity_outside_nucleus)

        if fgf_mean_intensities:
            binned_timepoints = np.array(timepoints)[::bin_size]
            binned_fgf_intensities = np.array(fgf_mean_intensities)[::bin_size]
            smoothed_fgf_intensities = gaussian_filter1d(binned_fgf_intensities, sigma=smoothing_sigma, mode='nearest')

            # Align curves so that all end at `max_timepoint`
            shift = max_timepoint - binned_timepoints[-1]
            aligned_timepoints = binned_timepoints + shift

            # Determine group color
            cell_group = cell_groups.get(cell_id, "mid_0")  
            color = group_colors.get(cell_group, "black")

            # Plot with one legend entry per group
            if cell_group not in group_labels:
                plt.plot(aligned_timepoints, smoothed_fgf_intensities, marker='.', linestyle='-', color=color, label=cell_group)
                group_labels.add(cell_group)
            else:
                plt.plot(aligned_timepoints, smoothed_fgf_intensities, marker='.', linestyle='-', color=color)

    plt.xlabel("Time (minutes)")
    plt.ylabel("FGF")
    plt.legend()
    plt.grid(True)
    plt.savefig(os.path.join(output_plot_folder, "Aligned_FGF_S1.svg"))
    plt.close()

    print("Processing and plotting complete!")

# Example function call:
input_nuclear_mask_folder = "E:/Agnese/20241212_fgfeGFP_h2a_modERK-rnanuclow/20241212_124318_Experiment/mastodon_tracking/all/nuclei/"
input_erk_folder = "E:/Agnese/20241212_fgfeGFP_h2a_modERK-rnanuclow/20241212_124318_Experiment/mastodon_tracking/all/erk/"
input_fgf_folder = "E:/Agnese/20241212_fgfeGFP_h2a_modERK-rnanuclow/20241212_124318_Experiment/mastodon_tracking/all/fgf/"
output_plot_folder = "E:/Agnese/20241212_fgfeGFP_h2a_modERK-rnanuclow/20241212_124318_Experiment/mastodon_tracking/all/results/"
selected_cell_ids = [1,2,8,9,13,3,5,6,7]
cell_groups = {
    **{cell: "anterior" for cell in [1, 2]},
    **{cell: "mid" for cell in [3,5,6,7]},
    **{cell: "posterior" for cell in [8,9,13]}
}

process_and_plot_fgf(input_nuclear_mask_folder, input_fgf_folder, output_plot_folder, selected_cell_ids, cell_groups, dilation_erosion_pixels=2, smoothing_sigma=3, bin_size=3)

Processing and plotting complete!
